# Calcul des KPI

In [37]:
import pandas as pd
import numpy as np
import os
import datetime
os.chdir("")

## Indication du fichier que l'on va utiliser

In [38]:
file_origine='BA_P_DP_TEST.csv'

## Lecture du fichier et intégration dans des Dataframes

In [39]:
#Lecture du fichier origine et mise dans un dataframe
df_origine=pd.read_csv(file_origine,delimiter=';', low_memory=False, encoding='latin-1')
#Affichage des 10 premières lignes du dataframe
df_origine.head(10)

,NUM,FR1_FECHA,FR1_LINEA,FR1_TIPO,FR1_REGIST,FR1_ORDEN,FR1_TREN,FR1_CAUC1,FR1_CAUC2,FR1_COCC1,...,Interrup_C1,Interval_Exc_C1,Desp_HP_C1_AM,Desp_HP_C1_PM,Desp_HV_C1,Interrup_C2,Interval_Exc_C2,Desp_HP_C2_AM,Desp_HP_C2_PM,Desp_HV_C2
0,0,2015-01-01,P,F,1,1,1,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
1,1,2015-01-01,P,F,2,2,1,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
2,2,2015-01-01,P,F,3,3,2,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
3,3,2015-01-01,P,F,4,4,3,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
4,4,2015-01-01,P,F,5,5,4,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
5,5,2015-01-01,P,F,6,6,5,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
6,6,2015-01-01,P,F,7,7,6,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
7,7,2015-01-01,P,F,8,8,1,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
8,8,2015-01-01,P,F,9,9,2,,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1
9,9,2015-01-01,P,F,10,10,3,V,,1,...,0 days 00:00:00.000000000,0,0,0,1,0 days 00:00:00.000000000,0,0,0,1


## Modification du type des colonnes contenant des dates / heures

In [40]:
#Convertion de la colonne au format date
df_origine['FR1_FECHA'] = pd.to_datetime(df_origine['FR1_FECHA'])

#Convertion de la colonne au format forcée de HH:MM:SS
df_origine['FR1_SALC1'] = pd.to_datetime(df_origine['FR1_SALC1'],format="%H:%M:%S",errors='ignore')
df_origine['FR1_SALC2'] = pd.to_datetime(df_origine['FR1_SALC2'],format="%H:%M:%S",errors='ignore')

#Convertion de la colonne au format forcée de HH:MM:SS
df_origine['SAL_C1'] = pd.to_datetime(df_origine['SAL_C1'],format="%H:%M:%S",errors='ignore')
df_origine['SAL_C2'] = pd.to_datetime(df_origine['SAL_C2'],format="%H:%M:%S",errors='ignore')

#On transforme au format interval de temps
df_origine['INT_C1'] = pd.to_timedelta(df_origine['INT_C1'])
df_origine['INT_C2'] = pd.to_timedelta(df_origine['INT_C2'])

#On transforme au format interval de temps
df_origine['I_REELC1'] = pd.to_timedelta(df_origine['I_REELC1'])
df_origine['I_REELC2'] = pd.to_timedelta(df_origine['I_REELC2'])

#On transforme au format interval de temps
df_origine['Interrup_C1'] = pd.to_timedelta(df_origine['Interrup_C1'])
df_origine['Interrup_C2'] = pd.to_timedelta(df_origine['Interrup_C2'])

df_origine.dtypes

NUM                          int64
FR1_FECHA           datetime64[ns]
FR1_LINEA                   object
FR1_TIPO                    object
FR1_REGIST                   int64
FR1_ORDEN                    int64
FR1_TREN                     int64
FR1_CAUC1                   object
FR1_CAUC2                   object
FR1_COCC1                    int64
FR1_COCC2                    int64
FR1_KM                      object
FR1_KMV                      int64
FR1_VIAC1                   object
FR1_VIAC2                   object
FR1_SALC1                   object
FR1_SALC2                   object
FR1_SCAUC2                  object
FR1_SCAUC1                  object
TIPO_DIA                    object
SAISON                      object
PERIODO                      int64
LINEA                       object
TREN                       float64
SAL_C1                      object
SAL_C2                      object
INT_C1             timedelta64[ns]
INT_C2             timedelta64[ns]
FORM                

## Synthèse des pénalités

# Calcul des pénalités mois par mois

## Calcul du KPI Interruption

In [41]:
#On utilise le nombre de voiture qui convient selon la ligne concernée 
coche = {'A': 5, 'B': 6, 'C': 6, 'D': 6, 'E': 5, 'H': 6, 'P': 1}
coche_result = coche.get(df_origine.iloc[1,2], 'default')

#On indexe le tableau origine selon la colonne qui contient des dates
df_origine.set_index(df_origine['FR1_FECHA'] ,inplace=True)

#Création du DataFrame de résultat
df_result=pd.DataFrame()
df_temp=pd.DataFrame()

#Somme des interruptions puis calcul du KPI interruption
df_temp['Sum_Interrup_C1'] = df_origine['Interrup_C1'].resample('D').sum()
df_temp['Sum_Interrup_C2'] = df_origine['Interrup_C2'].resample('D').sum()
df_temp['UP_Interrup']=((df_temp['Sum_Interrup_C1'].dt.seconds / 60) + (df_temp['Sum_Interrup_C2'].dt.seconds / 60)) * 25

df_result['UP_Interrup'] = df_temp['UP_Interrup'].resample('M').sum()

df_origine.head(10)

,NUM,FR1_FECHA,FR1_LINEA,FR1_TIPO,FR1_REGIST,FR1_ORDEN,FR1_TREN,FR1_CAUC1,FR1_CAUC2,FR1_COCC1,...,Interrup_C1,Interval_Exc_C1,Desp_HP_C1_AM,Desp_HP_C1_PM,Desp_HV_C1,Interrup_C2,Interval_Exc_C2,Desp_HP_C2_AM,Desp_HP_C2_PM,Desp_HV_C2
FR1_FECHA,,,,,,,,,,,,,,,,,,,,,
2015-01-01,0,2015-01-01,P,F,1,1,1,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,1,2015-01-01,P,F,2,2,1,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,2,2015-01-01,P,F,3,3,2,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,3,2015-01-01,P,F,4,4,3,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,4,2015-01-01,P,F,5,5,4,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,5,2015-01-01,P,F,6,6,5,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,6,2015-01-01,P,F,7,7,6,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,7,2015-01-01,P,F,8,8,1,V,,1,...,0 days,0,0,0,1,0 days,0,0,0,1
2015-01-01,8,2015-01-01,P,F,9,9,2,,,1,...,0 days,0,0,0,1,0 days,0,0,0,1


## Calcul du KPI Interval

In [42]:
#Nombre d'interval excédentaire
df_temp['Count_Interval_Exc_C1'] = df_origine['Interval_Exc_C1'].resample('D').sum()
df_temp['Count_Interval_Exc_C2'] = df_origine['Interval_Exc_C2'].resample('D').sum()

#Nombre d'interval theo
df_origine['INT_C1_count'] = np.where(df_origine['INT_C1']!=datetime.timedelta(seconds=1),1,0)
df_origine['INT_C2_count'] = np.where(df_origine['INT_C2']!=datetime.timedelta(seconds=1),1,0)
df_temp['Count_Interval_Theo_C1'] = df_origine['INT_C1_count'].resample('D').sum()
df_temp['Count_Interval_Theo_C2'] = df_origine['INT_C2_count'].resample('D').sum()

#Calcul de l'indice du KPI interval
df_temp['Indice_Interval'] = (df_temp['Count_Interval_Exc_C1'] + df_temp['Count_Interval_Exc_C2']) / (df_temp['Count_Interval_Theo_C1'] + df_temp['Count_Interval_Theo_C2'])

#Calcul du KPI interval
UP_Interval = 5 * (df_temp['Count_Interval_Exc_C1'] + df_temp['Count_Interval_Exc_C2']) - (0.03 * (df_temp['Count_Interval_Theo_C1'] + df_temp['Count_Interval_Theo_C2']))
df_temp['UP_Interval'] = np.where(df_temp['Indice_Interval']>0.03,UP_Interval,0)

df_result['UP_Interval'] = df_temp['UP_Interval'].resample('M').sum()

df_result.head(10)

,UP_Interrup,UP_Interval
FR1_FECHA,,
2015-01-31,19489.166667,3882.33
2015-02-28,10146.666667,3143.64
2015-03-31,18890.000000,3192.28
2015-04-30,6557.500000,3456.02
2015-05-31,12065.833333,4169.16
2015-06-30,14191.250000,3417.88
2015-07-31,20370.000000,4265.95
2015-08-31,22099.583333,4386.02
2015-09-30,31725.000000,3276.12


## Calcul du KPI départ HP

In [43]:
#Terminus
Cabecera_1 = {'A' : 'SAN PEDRITO', 'B' : 'JUAN MANUEL DE ROSAS', 'C' : 'CONSTITUCION', 'D' : 'CONGRESO DE TUCUMAN', 'E' : 'PLAZA DE LOS VIRREYES', 'H' : 'LAS HERAS', 'P' : 'INTENDENTE SAGUIER'}
Cabecera_2 = {'A' : 'PLAZA DE MAYO', 'B' : 'LEANDRO N.ALEM', 'C' : 'RETIRO', 'D' : 'CATEDRAL', 'E' : 'BOLIVAR', 'H' : 'HOSPITALES', 'P' : 'GRAL SAVIO'}

Sens_AM = {'A' : 'C1', 'B' : 'C1', 'C' : 'C1', 'D' : 'C1', 'E' : 'C1', 'H' : 'C2', 'P' : 'C1'}
Sens_PM = {'A' : 'C2', 'B' : 'C2', 'C' : 'C2', 'D' : 'C2', 'E' : 'C2', 'H' : 'C1', 'P' : 'C2'}

if df_origine.iloc[1,2] == 'H':
    #Nombre de depart Theo HP AM
    df_origine['Dep_Theo_C2_HP_AM'] = np.where(np.logical_and(df_origine['INT_C2']!=datetime.timedelta(seconds=1),df_origine['Desp_HP_C2_AM']==1),1,0)
    df_temp['Count_Depart_Theo_HP_AM'] = df_origine['Dep_Theo_C2_HP_AM'].resample('D').sum()
    
    #Nombre de depart réel HP AM
    df_origine['Dep_Reel_C2_HP_AM'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC2']),df_origine['Desp_HP_C2_AM']==1),1,0)
    df_temp['Count_Depart_reel_HP_AM'] = df_origine['Dep_Reel_C2_HP_AM'].resample('D').sum()
    
    #Calcul del'indice du KPI départ HP AM
    df_temp['Indice_Depart_HP_AM'] = df_temp['Count_Depart_reel_HP_AM'] / df_temp['Count_Depart_Theo_HP_AM']

    #Calcul du KPI depart HP AM
    UP_Depart_HP = 10 * (df_temp['Count_Depart_Theo_HP_AM'] - df_temp['Count_Depart_reel_HP_AM']) * coche_result
    df_temp['UP_Depart_HP_AM'] = np.where(df_temp['Indice_Depart_HP_AM']<0.97,UP_Depart_HP,0)

    df_result['UP_Depart_HP_AM'] = df_temp['UP_Depart_HP_AM'].resample('M').sum()
    
    #Nombre de depart Theo HP PM
    df_origine['Dep_Theo_C1_HP_PM'] = np.where(np.logical_and(df_origine['INT_C1']!=datetime.timedelta(seconds=1),df_origine['Desp_HP_C1_PM']==1),1,0)
    df_temp['Count_Depart_Theo_HP_PM'] = df_origine['Dep_Theo_C1_HP_PM'].resample('D').sum()
    
    #Nombre de depart réel HP PM
    df_origine['Dep_Reel_C1_HP_PM'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC1']),df_origine['Desp_HP_C1_PM']==1),1,0)
    df_temp['Count_Depart_reel_HP_PM'] = df_origine['Dep_Reel_C1_HP_PM'].resample('D').sum()
    
    #Calcul del'indice du KPI départ HP PM
    df_temp['Indice_Depart_HP_PM'] = df_temp['Count_Depart_reel_HP_PM'] / df_temp['Count_Depart_Theo_HP_PM']

    #Calcul du KPI depart HP PM
    UP_Depart_HP = 10 * (df_temp['Count_Depart_Theo_HP_PM'] - df_temp['Count_Depart_reel_HP_PM']) * coche_result
    df_temp['UP_Depart_HP_PM'] = np.where(df_temp['Indice_Depart_HP_PM']<0.97,UP_Depart_HP,0)

    df_result['UP_Depart_HP_PM'] = df_temp['UP_Depart_HP_PM'].resample('M').sum()
    
else:
    #Nombre de depart Theo HP AM
    df_origine['Dep_Theo_C1_HP_AM'] = np.where(np.logical_and(df_origine['INT_C1']!=datetime.timedelta(seconds=1),df_origine['Desp_HP_C1_AM']==1),1,0)
    df_temp['Count_Depart_Theo_HP_AM'] = df_origine['Dep_Theo_C1_HP_AM'].resample('D').sum()
    
    #Nombre de depart réel HP AM
    df_origine['Dep_Reel_C1_HP_AM'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC1']),df_origine['Desp_HP_C1_AM']==1),1,0)
    df_temp['Count_Depart_reel_HP_AM'] = df_origine['Dep_Reel_C1_HP_AM'].resample('D').sum()
    
    #Calcul del'indice du KPI départ HP AM
    df_temp['Indice_Depart_HP_AM'] = df_temp['Count_Depart_reel_HP_AM'] / df_temp['Count_Depart_Theo_HP_AM']

    #Calcul du KPI depart HP AM
    UP_Depart_HP = 10 * (df_temp['Count_Depart_Theo_HP_AM'] - df_temp['Count_Depart_reel_HP_AM']) * coche_result
    df_temp['UP_Depart_HP_AM'] = np.where(df_temp['Indice_Depart_HP_AM']<0.97,UP_Depart_HP,0)

    df_result['UP_Depart_HP_AM'] = df_temp['UP_Depart_HP_AM'].resample('M').sum()
    
    #Nombre de depart Theo HP PM
    df_origine['Dep_Theo_C2_HP_PM'] = np.where(np.logical_and(df_origine['INT_C2']!=datetime.timedelta(seconds=1),df_origine['Desp_HP_C2_PM']==1),1,0)
    df_temp['Count_Depart_Theo_HP_PM'] = df_origine['Dep_Theo_C2_HP_PM'].resample('D').sum()
    
    #Nombre de depart réel HP PM
    df_origine['Dep_Reel_C2_HP_PM'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC2']),df_origine['Desp_HP_C2_PM']==1),1,0)
    df_temp['Count_Depart_reel_HP_PM'] = df_origine['Dep_Reel_C2_HP_PM'].resample('D').sum()
    
    #Calcul del'indice du KPI départ HP PM
    df_temp['Indice_Depart_HP_PM'] = df_temp['Count_Depart_reel_HP_PM'] / df_temp['Count_Depart_Theo_HP_PM']

    #Calcul du KPI depart HP PM
    UP_Depart_HP = 10 * (df_temp['Count_Depart_Theo_HP_PM'] - df_temp['Count_Depart_reel_HP_PM']) * coche_result
    df_temp['UP_Depart_HP_PM'] = np.where(df_temp['Indice_Depart_HP_PM']<0.97,UP_Depart_HP,0)

    df_result['UP_Depart_HP_PM'] = df_temp['UP_Depart_HP_PM'].resample('M').sum()
    
df_result['UP_Depart_HP'] = df_result['UP_Depart_HP_AM'] + df_result['UP_Depart_HP_PM']

df_result.head(10)

,UP_Interrup,UP_Interval,UP_Depart_HP_AM,UP_Depart_HP_PM,UP_Depart_HP
FR1_FECHA,,,,,
2015-01-31,19489.166667,3882.33,910,0,910
2015-02-28,10146.666667,3143.64,770,0,770
2015-03-31,18890.000000,3192.28,1070,0,1070
2015-04-30,6557.500000,3456.02,1130,0,1130
2015-05-31,12065.833333,4169.16,1060,0,1060
2015-06-30,14191.250000,3417.88,1500,0,1500
2015-07-31,20370.000000,4265.95,1350,0,1350
2015-08-31,22099.583333,4386.02,1360,0,1360
2015-09-30,31725.000000,3276.12,1670,0,1670


## Calcul du KPI départ HV

In [44]:
#Nombre de depart Theo HV
df_origine['Dep_Theo_C1_HV'] = np.where(np.logical_and(df_origine['INT_C1']!=datetime.timedelta(seconds=1),df_origine['Desp_HV_C1']==1),1,0)
df_origine['Dep_Theo_C2_HV'] = np.where(np.logical_and(df_origine['INT_C2']!=datetime.timedelta(seconds=1),df_origine['Desp_HV_C2']==1),1,0)
df_temp['Count_Depart_Theo_HV_C1'] = df_origine['Dep_Theo_C1_HV'].resample('D').sum()
df_temp['Count_Depart_Theo_HV_C2'] = df_origine['Dep_Theo_C2_HV'].resample('D').sum()

#Nombre de depart réel HV
df_origine['Dep_Reel_C1_HV'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC1']),df_origine['Desp_HV_C1']==1),1,0)
df_origine['Dep_Reel_C2_HV'] = np.where(np.logical_and(pd.notnull(df_origine['I_REELC2']),df_origine['Desp_HV_C2']==1),1,0)
df_temp['Count_Depart_reel_HV_C1'] = df_origine['Dep_Reel_C1_HV'].resample('D').sum()
df_temp['Count_Depart_reel_HV_C2'] = df_origine['Dep_Reel_C2_HV'].resample('D').sum()

#Calcul de l'indice du KPI départ HV
df_temp['Indice_Depart_HV'] = (df_temp['Count_Depart_reel_HV_C1'] + df_temp['Count_Depart_reel_HV_C2']) / (df_temp['Count_Depart_Theo_HV_C1'] + df_temp['Count_Depart_Theo_HV_C2'])

#Calcul du KPI depart HV
UP_Depart_HV = 5 * ((df_temp['Count_Depart_Theo_HV_C1'] + df_temp['Count_Depart_Theo_HV_C2']) - (df_temp['Count_Depart_reel_HV_C1'] + df_temp['Count_Depart_reel_HV_C2'])) * coche_result
df_temp['UP_Depart_HV'] = np.where(df_temp['Indice_Depart_HV']<0.97,UP_Depart_HV,0)

df_result['UP_Depart_HV'] = df_temp['UP_Depart_HV'].resample('M').sum()

df_result.head(10)

,UP_Interrup,UP_Interval,UP_Depart_HP_AM,UP_Depart_HP_PM,UP_Depart_HP,UP_Depart_HV
FR1_FECHA,,,,,,
2015-01-31,19489.166667,3882.33,910,0,910,4130
2015-02-28,10146.666667,3143.64,770,0,770,3685
2015-03-31,18890.000000,3192.28,1070,0,1070,4030
2015-04-30,6557.500000,3456.02,1130,0,1130,3255
2015-05-31,12065.833333,4169.16,1060,0,1060,4405
2015-06-30,14191.250000,3417.88,1500,0,1500,4380
2015-07-31,20370.000000,4265.95,1350,0,1350,4215
2015-08-31,22099.583333,4386.02,1360,0,1360,5395
2015-09-30,31725.000000,3276.12,1670,0,1670,6430


## Calcul du total et affichage des résultats

In [45]:
#Calcul UP total
df_result['UP_total'] = df_result['UP_Interrup'] + df_result['UP_Interval'] + df_result['UP_Depart_HP'] + df_result['UP_Depart_HV']

df_result.head(50)
#df_result.dtypes

print(df_result['UP_total'].sum())

914015.3333333334


## Calcul du nombre d'incident par catégorie de pénalités

In [46]:
df_synthese = pd.DataFrame()
df_temp2 = pd.DataFrame()

df_origine['Nb_incident_Interrup_C1'] = np.where(df_origine['Interrup_C1']!=datetime.timedelta(seconds=0),1,0)
df_origine['Nb_incident_Interrup_C2'] = np.where(df_origine['Interrup_C2']!=datetime.timedelta(seconds=0),1,0)
df_synthese['Nb_incident_Interrup'] = df_origine['Nb_incident_Interrup_C1'].resample('M').sum() + df_origine['Nb_incident_Interrup_C2'].resample('M').sum()

df_synthese['Nb_incident_Interval'] = df_temp['Count_Interval_Exc_C1'].resample('M').sum() + df_temp['Count_Interval_Exc_C2'].resample('M').sum()

df_temp2['Nb_incident_depart_HP_AM'] = df_temp['Count_Depart_Theo_HP_AM'].resample('D').sum() - df_temp['Count_Depart_reel_HP_AM'].resample('D').sum()
df_temp2['Nb_incident_depart_HP_AM'] = np.where(df_temp2['Nb_incident_depart_HP_AM']<0,0,df_temp2['Nb_incident_depart_HP_AM'])
df_synthese['Nb_incident_depart_HP_AM'] = df_temp2['Nb_incident_depart_HP_AM'].resample('M').sum()

df_temp2['Nb_incident_depart_HP_PM'] = df_temp['Count_Depart_Theo_HP_PM'].resample('D').sum() - df_temp['Count_Depart_reel_HP_PM'].resample('D').sum()
df_temp2['Nb_incident_depart_HP_PM'] = np.where(df_temp2['Nb_incident_depart_HP_PM']<0,0,df_temp2['Nb_incident_depart_HP_PM'])
df_synthese['Nb_incident_depart_HP_PM'] = df_temp2['Nb_incident_depart_HP_PM'].resample('M').sum()

df_synthese['Nb_incident_depart_HP'] = df_synthese['Nb_incident_depart_HP_AM'] + df_synthese['Nb_incident_depart_HP_PM']

df_temp2['Nb_incident_depart_HV'] = (df_temp['Count_Depart_Theo_HV_C1'].resample('D').sum() + df_temp['Count_Depart_Theo_HV_C2'].resample('D').sum()) - (df_temp['Count_Depart_reel_HV_C1'].resample('D').sum() + df_temp['Count_Depart_reel_HV_C2'].resample('D').sum())
df_temp2['Nb_incident_depart_HV'] = np.where(df_temp2['Nb_incident_depart_HV']<0,0,df_temp2['Nb_incident_depart_HV'])
df_synthese['Nb_incident_depart_HV'] = df_temp2['Nb_incident_depart_HV'].resample('M').sum()

#Export en CSV
export_tot='BA_' + df_origine.iloc[1,2] + '_DP_nb_incident.csv'
print(export_tot)
df_synthese.to_csv(export_tot,sep=';',index=True, decimal=',')

df_synthese.head (36)

BA_P_DP_nb_incident.csv


,Nb_incident_Interrup,Nb_incident_Interval,Nb_incident_depart_HP_AM,Nb_incident_depart_HP_PM,Nb_incident_depart_HP,Nb_incident_depart_HV
FR1_FECHA,,,,,,
2015-01-31,74,798,91,0,91,826
2015-02-28,39,648,77,0,77,737
2015-03-31,50,659,107,0,107,806
2015-04-30,36,712,113,0,113,651
2015-05-31,70,855,106,0,106,881
2015-06-30,47,704,150,0,150,876
2015-07-31,78,875,135,0,135,843
2015-08-31,95,898,136,0,136,1079
2015-09-30,89,675,167,0,167,1286


## Export des résultats

In [47]:
#Export en CSV
export_tot='BA_' + df_origine.iloc[1,2] + '_DP_Result.csv'
print(export_tot)
df_result.to_csv(export_tot,sep=';',index=True, decimal=',')

#Export en Excel
writer = pd.ExcelWriter('BA_' + df_origine.iloc[1,2] + '_Result.xlsx')
df_origine.to_excel(writer,'Data')
df_temp.to_excel(writer,'Jours')
df_result.to_excel(writer,'Mois')
writer.save()

BA_P_DP_Result.csv


## Test de graphique

In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode

#print(__version__)# requires version >= 1.9.0

import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

df_graph = df_result

df_graph.reset_index(inplace=True)

trace = go.Bar(x=df_result['FR1_FECHA'],
               y=df_result['UP_total'],
               name='Unités de pénalités')

axis_template = dict(title='Buenos Aires - Ligne ' + df_origine.iloc[1,2] + ' - Pénalités')

layout = go.Layout(showlegend=True,
                   yaxis=axis_template)

data = [trace]

fig = go.Figure(data=data,layout=layout)

plotly.offline.plot(fig)

ModuleNotFoundError: No module named 'plotly'

## Statistiques automatiques via pandas-profiling